In [135]:
# Code for data collection
from bs4 import BeautifulSoup as bs
import requests as rq
import re

langs = {"javascript": [], " c[ \.]": [], "c\+\+": [], "java": [], "kotlin": [], "swift": [], "ruby": [], "python3?": [], "sql": [], " r[ \.]": [], "perl": [], "c#": [], "lua": [], "rust": [], "php": [], "html": [], "css": [], "xml": []}

page = rq.get("https://www.linkedin.com/jobs/search?keywords=Software%2Bengineer&location=United%20States&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")
soup = bs(page.content, 'html.parser')
items = soup.find_all(class_="base-card__full-link")
print(langs.values())
salary_match = "\$?\d{0,3},?\d{1,3},\d{3,3}\.?\d{0,2}"
item = items[0]
for item in items:
    url = item.get('href')
    print(url + "\n")
    posting_page = rq.get(url)
    s = bs(posting_page.content, 'html.parser')
    text = s.get_text()
    sal_matches = list(map(lambda x: float(re.sub("[^0-9\.]", "", x)), re.findall(salary_match, text)))
    for k, v in langs.items(): 
        if re.search(k, text, flags=re.IGNORECASE) != None:
            langs[k].extend(sal_matches)
    
print(langs)

In [ ]:
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller
import re

# WHEN SELENIUM IS RUNNING, DON'T CLICK AROUND OTHER TABS, THIS WILL BREAK PROGRAM
def main():
    langs = {"javascript": [], " c[ \.]": [], "c\+\+": [], "java": [], "kotlin": [], "swift": [], "ruby": [], "python3?": [], "sql": [], " r[ \.]": [], "perl": [], "c#": [], "lua": [], "rust": [], "php": [], "html": [], "css": [], "xml": []}
    keyboard = Controller()
    
    # Initialize Driver
    url = 'https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm'
    driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
    driver.get(url)     
    
    num_pages_to_search = 0
    salary_match = "\$?\d{0,3},?\d{1,3},\d{3,3}\.?\d{0,2}"
    
    # extract the job data
    while num_pages_to_search < 2:
        # Each job
        jobs_on_page = driver.find_elements_by_class_name('react-job-listing')
        for job_card in jobs_on_page:
            job_card.click()
            # sleep to load the info 
            sleep(0.4)
            # escape any popups
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)
            try:
                text = driver.find_element_by_class_name('jobDescriptionContent').text.lower()
                salary = process_salary(driver.find_element_by_class_name('css-y2jiyn').text)
#                 sal_matches = list(map(lambda x: float(re.sub("[^0-9\.]", "", x)), re.findall(salary_match, text)))
                for k, v in langs.items(): 
                    if re.search(k, text, flags=re.IGNORECASE) != None:
                        print("HAS SAL")
                        langs[k].append(salary)
#                 print("Salary: " + str(salary))
#                 for language in langs:
#                     if language in job_description:
#                         langs[language].append(salary)
            except NoSuchElementException:
                print("No salary provided")
        num_pages_to_search += 1
        print(langs)
        # go to next page
        driver.find_element_by_class_name('nextButton').click()
        sleep(2)

# $70,000 /yr (est.) => 70000
def process_salary(salary):
    new_salary = salary.split(' ', 1)
    if new_salary[1] == '/yr (est.)':
        return new_salary[0][1:]
    else:
        return float(new_salary[0][1:]) * 52 * 40
        
main()




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/eddiexu/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache


HAS SAL
HAS SAL
HAS SAL
HAS SAL
No salary provided
No salary provided
HAS SAL
HAS SAL
HAS SAL
HAS SAL
HAS SAL
HAS SAL
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
HAS SAL
HAS SAL
{'javascript': ['90,451', '122,269', 156000.0], ' c[ \\.]': [], 'c\\+\\+': [], 'java': ['90,451', '134,000', '122,269', 156000.0], 'kotlin': [], 'swift': [], 'ruby': ['106,018'], 'python3?': ['90,451', '134,000'], 'sql': ['134,000', '122,269'], ' r[ \\.]': [], 'perl': [], 'c#': [], 'lua': [], 'rust': [], 'php': [], 'html': [], 'css': [], 'xml': []}
HAS SAL
HAS SAL
HAS SAL
HAS SAL
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
No salary provided
HAS SAL
HAS SAL
HAS SAL
HAS SAL
HAS SAL
No salary provided
HAS SAL
HAS SAL
HAS SAL
HAS SAL
HAS SAL
HAS SAL
HAS SAL
HAS SAL
No salary provided
No salary provided

In [119]:
import csv
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller

def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url
def save_data_to_file(records):
    """Save data to csv file"""
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'JobUrl'])
        writer.writerows(records)
def get_record(card):
    """Extract job data from single card"""
    job_title = card.find_element_by_class_name('jobtitle').text
    company = card.find_element_by_class_name('company').text
    location = card.find_element_by_class_name('location').text
    post_date = card.find_element_by_class_name('date').text
    extract_date = datetime.today().strftime('%Y-%m-%d')
    summary = card.find_element_by_class_name('summary').text
    job_url = card.find_element_by_class_name('jobtitle').get_attribute('href')
    return (job_title, company, location, post_date, extract_date, summary, job_url)
def get_page_records(cards, job_list, url_set):
    """Extract all cards from the page"""
    for card in cards:
        record = get_record(card)
        # add if job title exists and not duplicate
        if record[0] and record[-1] not in url_set:
            job_list.append(record)
            url_set.add(record[-1])
def main(position, location):
    """Run the main program routine"""
    scraped_jobs = []
    scraped_urls = set()
    
    url = get_url(position, location)
    
    # setup web driver
    driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
    driver.get(url)
    # extract the job data
    while True:
        cards = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
        get_page_records(cards, scraped_jobs, scraped_urls)
        try:
            driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
        except NoSuchElementException:
            break
        except ElementNotInteractableException:
            driver.find_element_by_id('popover-x').click()  # to handle job notification popup
            get_page_records(cards, scraped_jobs, scraped_urls)
            continue
    
    # close driver and save records
    driver.quit()
    save_data_to_file(scraped_jobs)
# run a search
main('python developer', 'charlotte nc')



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/eddiexu/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache


[]
dict_values([[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []])
{'javascript': [], ' c[ \\.]': [], 'c\\+\\+': [], 'java': [], 'kotlin': [], 'swift': [], 'ruby': [], 'python3?': [], 'sql': [], ' r[ \\.]': [], 'perl': [], 'c#': [], 'lua': [], 'rust': [], 'php': [], 'html': [], 'css': [], 'xml': []}
